# Lab 4: Problem set on Newton-Cotes formulas

The main task of this problem set is the following: for a given interval $(a,b)$ and $n+1$ equally distributed nodes $x_i = a + i \tfrac{(b-a)}{n}$
for $i=0,\ldots n$, tabulate the weights for the Newton-Cotes formula up to $n=14$.
The problem boils down to 2 task, namely 

1. Defining the Lagrange polynomials $L_{in}$ for $i=0, \ldots, n$
2. Computing the weights $ w_i = \int_a^b L_{in}(x) \,\mathrm{d}x $

__Before you start:__ For the implementation of the task, we recommend to use the [sympy](https://docs.sympy.org/latest/index.html#) python module  for symbolic mathematics to perform tasks such as (symbolic) integration.
Spend some time to browse  through the  [sympy tutorial](https://docs.sympy.org/latest/tutorial/index.html) and the [help on symbolic integration](https://docs.sympy.org/latest/modules/integrals/integrals.html). Using ```sympy```__Before you start:__ For the implementation of the task, we recommend to use the [sympy](https://docs.sympy.org/latest/index.html#) python module  for symbolic mathematics to perform tasks such as (symbolic) integration.
Spend some time to browse  through the  [sympy tutorial](https://docs.sympy.org/latest/tutorial/index.html) and the [help on symbolic integration](https://docs.sympy.org/latest/modules/integrals/integrals.html). Using ```sympy``` you can for instance do something like this for $n=2$:


In [1]:
# import symbol x from sympy so that you can define symbolic functions of x
from sympy.abc import x
# import symbolic integration
from sympy import integrate 
import numpy as np
from functools import reduce

# Define
a, b = 0, 1 
xqs = np.linspace(a,b,3)
# Define L_02 (not normalized)
L_02 = (x-xqs[1])*(x-xqs[2])
# Normalize it to satisfy l_02(x_0) = 1
L_02 = L_02/L_02.subs(x,xqs[0])

# Now integrate L_02 to compute the first weight
w_0 = integrate(L_02, (x, a, b))
print("w0 = {}".format(w_0))

w0 = 0.166666666666667


Of course, 
since you are asked to do comupute all $n+1$ weights  $n=1,\ldots,14$,
you need to automatize the construction of the corresponding Lagrange polynoms.
So proceed as follows


__a__) Write  a python function ```lagrange_polys``` which takes a list of $n+1$ quadrature points
and returns a list of the corresponding $n+1$ Lagrange polynoms $\{L_{in}\}_{i=0}^n$
defined a symbolic function using ```sympy```:

In [ ]:
def lagrange_polys(xqs):
    n = len(xqs)
    # Define non-normalized Lagrange basis functions
    Ls = [reduce(lambda x,y : x*y,[x - xqs[j] for j in range(n) if j is not i]) for i in range(n)]
    # Normalize them
    Ls = [Ls[i]/Ls[i].subs(x, xqs[i]) for i in range(n)]
    return Ls


__b__) Now the easy part! Employ your brand new ```def lagrange_polys``` function and implement a python function which takes as argument the desired degree of exactness $n$ and the interval end points
$a,b$ and returns a list of quadrature points $\{x_i\}_{i=0}^n$ and quadrature weights $\{w_i\}_{i=0}^n$:

In [ ]:
def newton_cotes_formula(n, a, b):
    xqs = np.linspace(a,b,n+1)
    Ls = lagrange_polys(xqs)
    ws = np.array([integrate(L, (x, a, b)) for L in Ls ])
    return (xqs, ws)

__c__) Before you tabulate the quadrature weights with you newly implemented function, make sure that you implement them correctly. More, specifically, check for $n=1,\ldots 14$
that the computed Newton-Cotes formula integrates polynomials up to order $n$ __exactly__.

For $n$ is even, check that the corresponding Newton-Cotes rules even integrate polynomials up to order $n+1$  exactly (and not only up to $n$).

Note:  Due to floating point related errors and some numerical instabilities when computing
higher order Lagrange polynomials and integrals, the difference between the exact integral 
and the numerically error won't be 0, but around the machine precision for $n=1,2$ and then
for each increase of the order $n$ you will roughly loose of significant digit in
the difference between the two.

It might be useful to implement a little function ```qr``` first,
which takes $f$, $\{x_i\}_{i=0}^n$ and $\{w_i\}_{i=0}^n$ and
applies the corresponding quadrature to compute $\int_a^b f \,\mathrm{d}x$ numerically.


In [ ]:
def qr(f, xqs, ws):
    qr_f = np.array([w*f.subs(x, xq) for xq,w in zip(xqs, ws)]).sum()
    return qr_f

In [ ]:
mono = lambda x,m: x**m
print(mono(x,2))

for n in range(1,15):
    xqs, ws = newton_cotes_formula(n, a, b)
    print("Newton-Cotes rules for n = {}".format(n))
    for m in range(n+1):
        f = mono(x,m)
        int_f = integrate(f, (x,a,b))
        qr_f = qr(f, xqs, ws)
        print("integral(x**{})= {}".format(m,int_f))
        print("qr(x**{})= {}".format(m,qr_f))
        print("int - qr = {}".format(int_f - qr_f))


__d__) Tabulate the quadrature weights for the Newton-Cotes rule for $n=1,\ldots 14$. For which $n$  should you
refrain from using the resulting quadrature rule (and why?)

In [ ]:
for n in range(1,15):
    xqs, weights = newton_cotes_formula(n, a, b)
    neg_weights = weights[weights < 0]
    if len(neg_weights) > 0:
        print("Newton-Cotes formula for n = %d contains negative weights!\n Don't use it!" % n)

__e__) Finally, since we have all the nice machinery in place, we take a little extra-tour
and investigate the convergence of the Newton-Cotes rules for $n\to \infty$.

More precisely, compute for $f(x) = \cos(x)$ the integral $\int_{-4}^{5} f(x) \,\mathrm{d}x $ first analytically 
and then numerically using the Newton-Cotes rules for $n=1,\ldots, 14$ and tabulate
the quadrature error $E_n(f) = \int_{-4}^{5} \cos(x)\,\mathrm{d}x - \mathrm{NCR}(\cos, n)$.

Finally, repeat the same experiment for for $f(x) = \tfrac{1}{1+x^2}$.

